## Importing Libraries

In [6]:
import cv2
import cvzone
import numpy as np

#### Creating OpenCV window for object detection 

In [2]:
import cv2
import numpy as np

cap = cv2.VideoCapture("D:\\o-source\\GSSoC'24\\UAV Car Detection\\Dataset\\traffic.mp4")

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

algo = cv2.createBackgroundSubtractorMOG2()

min_width_rect = 20
min_height_rect = 20

while True:
    success, frame = cap.read()
    if not success:
        print("Video analysis successful")
        break
    
    grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey, (3, 3), 5)
    
    img_sub = algo.apply(blur)
    dilat = cv2.dilate(img_sub, np.ones((5, 5), np.uint8))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    dil = cv2.morphologyEx(dilat, cv2.MORPH_CLOSE, kernel)
    dil = cv2.morphologyEx(dil, cv2.MORPH_CLOSE, kernel)
    
    contours, _ = cv2.findContours(dil, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contours:
        (x, y, w, h) = cv2.boundingRect(c)
        validate_counter = (w >= min_width_rect) and (h >= min_height_rect)
        if not validate_counter:
            continue
        
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(frame, "Car Detected", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    cv2.imshow("Cars", frame)
    cv2.imshow("Processed", dil)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()